# COMM7380 Recommender Systems for Digital Media


## Funk SVD implementation

`funk-svd` is a Python 3 library implemented by [Geoffrey Bolmier](https://github.com/gbolmier/funk-svd) under MIT license. This is a fast version of the famous SVD algorithm popularized by Simon Funk [(here)](http://sifter.org/simon/journal/20061211.html) during the [Neflix Prize](http://en.wikipedia.org/wiki/Netflix_Prize) competition.

[`Numba`](http://numba.pydata.org/) is used to speed up the algorithm.


## Funk SVD for recommendation in a nutshell

We have a huge sparse matrix:

$$
R = \begin{pmatrix} {\color{Red} ?} & 2 & \cdots & {\color{Red} ?} & {\color{Red} ?} \\ {\color{Red} ?} & {\color{Red} ?} & \cdots & {\color{Red} ?} & 4.5 \\ \vdots & \ddots & \ddots & \ddots & \vdots \\ 3 & {\color{Red} ?} & \cdots & {\color{Red} ?} & {\color{Red} ?} \\ {\color{Red} ?} & {\color{Red} ?} & \cdots & 5 & {\color{Red} ?} \end{pmatrix}
$$
storing known ratings for a set of users and items:

$$u = 1, ..., U$$

$$i = 1, ..., I$$

The idea is to estimate unknown ratings by factorizing the rating matrix into two smaller matrices representing user and item characteristics:

$$
P = \begin{pmatrix} 0.37 & \cdots & 0.69 \\ \vdots & \ddots & \vdots \\ \vdots & \ddots & \vdots \\ \vdots & \ddots & \vdots \\ 1.08 & \cdots & 0.24 \end{pmatrix} , Q = \begin{pmatrix} 0.09 & \cdots & \cdots & \cdots & 0.46 \\ \vdots & \ddots & \ddots & \ddots & \vdots \\ 0.51 & \cdots & \cdots & \cdots & 0.72 \end{pmatrix}
$$

We call these two matrices users and items latent factors. Then, by applying the dot product between both matrices we can reconstruct our rating matrix. 

NOTE: Using this method the empty values will contain estimated ratings.

In order to get more accurate results, the global average rating as well as the user and item biases are used in addition:

$$\bar{r} = \frac{1}{N} \sum_{i=1}^{N} K_{i}$$

where K stands for known ratings.

$$bu = \begin{pmatrix} 0.35 & \cdots & 0.07 \end{pmatrix}$$

$$bi = \begin{pmatrix} 0.16 & \cdots & 0.40 \end{pmatrix}$$

Then, we can estimate any rating by applying:

$$\hat{r}_{u, i} = \bar{r} + bu_{u} + bi_{i} + \sum_{f=1}^{F} P_{u, f} * Q_{i, f}$$

The learning step consists in performing the SGD algorithm where for each known rating the biases and latent factors are updated as follows:

$$err = r - \hat{r}$$

$$bu_{u} = bu_{u} + \alpha * (err - \lambda * bu_{u})$$

$$bi_{i} = bi_{i} + \alpha * (err - \lambda * bi_{i})$$

$$P_{u, f} = P_{u, f} + \alpha * (err * Q_{i, f} - \lambda * P_{u, f})$$

$$Q_{i, f} = Q_{i, f} + \alpha * (err * P_{u, f} - \lambda * Q_{i, f})$$

where $\alpha$ is the learning rate and $\lambda$ is the regularization term.

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install numba

In [1]:
import pandas as pd
import numpy as np

from funk_svd.dataset import fetch_ml_ratings
from funk_svd import SVD

from sklearn.metrics import mean_absolute_error

In [2]:
# Download dataset
df = fetch_ml_ratings(variant='100k')

# Create training/validation/test sets
train = df.sample(frac=0.8, random_state=57) # 80% of dataset
val = df.drop(train.index.tolist()).sample(frac=0.5, random_state=42) # 50% of training set
test = df.drop(train.index.tolist()).drop(val.index.tolist()) # remaining 20% of dataset

In [3]:
# Number of users
len(df['u_id'].unique())

943

In [4]:
# Number of items
len(df['i_id'].unique())

1682

In [5]:
# Size of User-Item Matrix
ui_size = len(df['u_id'].unique()) * len(df['i_id'].unique())
ui_size

1586126

In [6]:
# Number of ratings in dataset
n_ratings = df.shape[0]
n_ratings

99999

In [7]:
print('ratings-matrix size ratio: ', n_ratings / ui_size)

ratings-matrix size ratio:  0.06304606317530889


In [8]:
# Print a sample of dataset
train.head(5)

,u_id,i_id,rating,timestamp
28544,459,289,4.0,1997-11-15 10:41:19
72251,327,405,2.0,1998-02-18 03:59:49
65977,537,89,4.0,1998-01-29 07:41:02
28936,535,429,3.0,1997-11-16 02:29:29
21256,562,443,5.0,1997-11-11 05:16:44


In [15]:
# Use Funk SVD 
svd = SVD(learning_rate=0.001, regularization=0.005, n_epochs=100,
          n_factors=15, min_rating=1, max_rating=5)

In [16]:
svd.fit(X=train, X_val=val, early_stopping=True, shuffle=False)

Preprocessing data...

Epoch 1/100  | val_loss: 1.17 - val_rmse: 1.08 - val_mae: 0.90 - took 0.8 sec
Epoch 2/100  | val_loss: 1.10 - val_rmse: 1.05 - val_mae: 0.87 - took 0.0 sec
Epoch 3/100  | val_loss: 1.06 - val_rmse: 1.03 - val_mae: 0.85 - took 0.0 sec
Epoch 4/100  | val_loss: 1.04 - val_rmse: 1.02 - val_mae: 0.83 - took 0.0 sec
Epoch 5/100  | val_loss: 1.02 - val_rmse: 1.01 - val_mae: 0.81 - took 0.0 sec
Epoch 6/100  | val_loss: 1.00 - val_rmse: 1.00 - val_mae: 0.80 - took 0.0 sec
Epoch 7/100  | val_loss: 0.99 - val_rmse: 0.99 - val_mae: 0.80 - took 0.0 sec
Epoch 8/100  | val_loss: 0.98 - val_rmse: 0.99 - val_mae: 0.79 - took 0.0 sec
Epoch 9/100  | val_loss: 0.97 - val_rmse: 0.98 - val_mae: 0.79 - took 0.0 sec
Epoch 10/100 | val_loss: 0.96 - val_rmse: 0.98 - val_mae: 0.78 - took 0.0 sec
Epoch 11/100 | val_loss: 0.96 - val_rmse: 0.98 - val_mae: 0.78 - took 0.0 sec
Epoch 12/100 | val_loss: 0.95 - val_rmse: 0.98 - val_mae: 0.78 - took 0.0 sec
Epoch 13/100 | val_loss: 0.95 - val_rmse:

In [17]:
df[(df['u_id'] == 459) & (df['i_id'] == 290)]

,u_id,i_id,rating,timestamp


In [18]:
svd.predict_pair(459, 290)

3.2629941971227336

- Course Instructor: Dr. Paolo Mengoni (Visiting Scholar, School of Communication, Hong Kong Baptist University) 
  - pmengoni@hkbu.edu.hk

- The codes in this notebook take insipiration from various sources. All codes are for educational purposes only and released under the CC1.0. 
- Funk SVD code is released by the author under MIT license.